# Cherry-picked Images

In [1]:
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image

In [2]:
task = 'strokegen'

In [ ]:
_METHODS_FACE = {
    'BoN' : f'bon_40_{task}',
    'DPS (2023)' : f'grad200_{task}' if task == 'facedetector' else f'grad13_{task}',
    'UG (2024)': f'test_face_20000' if task == 'facedetector' else 'test_style_6' if task == 'styletransfer' else 'test_strokegen_6',
    'SVDD-PM (2024)': f'code40_b1_{task}',
    'CoDe (Ours)' : f'code40_b5_{task}' if task == 'strokegen' else f'code100_b5_{task}',
    'C-CoDe (Ours)' : f'c_code40_b5_r6_{task}' if task == 'strokegen' else f'c_code100_b5_r6_{task}' if task == 'styletransfer' else f'c_code100_b5_r7_{task}',
}

if task == 'facedetector':
    target_path = Path('D:\\TempFiles\\ICLR_CoDe\\face_data\\celeb')
    target_map = {
        'og_img_4': 0,
        'og_img_6': 1,
        'og_img_8': 2,
    }
elif task == 'styletransfer':       
    target_path = Path('D:\\TempFiles\ICLR_CoDe\\style_folder\\styles')
    target_map = {
        'style_0': 0,
        'style_1': 1,
        'style_2': 2,
    }
else:
    target_path = Path('D:\\TempFiles\\ICLR_CoDe\\stroke_gen')
    target_map = {
        'stroke_img_0': 0,
        'stroke_img_1': 1,
        'stroke_img_3': 2,
    }

base_path = Path('D:\\TempFiles\\ICLR_CoDe\\cherry_picking_human\\cherry_picking_human')

save_path = Path(f'D:\\TempFiles\\ICLR_CoDe\\{task}')

target_dirs = [x for x in save_path.iterdir() if Path.is_dir(x)]
for target_dir in target_dirs:

    if 'stroke_img_3' not in target_dir.stem:
        continue

    prompt_dirs = [x for x in target_dir.joinpath('images').iterdir() if Path.is_dir(x)]
    for prompt_dir in prompt_dirs:

        plt_save = save_path.joinpath(target_dir.stem).joinpath('images').joinpath(prompt_dir.stem)
        # if Path.exists(plt_save.joinpath('res2.png')):
        #     continue

        fig, ax = plt.subplots(5, 7, figsize=(20,15))

        if task == 'styletransfer':
            target_path = Path('D:\TempFiles\ICLR_CoDe\style_folder\styles')
            target = Image.open(target_path.joinpath(f'{target_dir.stem}.jpg'))
        elif task == 'facedetector':
            target_path = Path('D:\\TempFiles\\ICLR_CoDe\\face_data\\celeb')
            target = Image.open(target_path.joinpath(f'{target_dir.stem}.png'))
        else:
            target_path = Path('D:\\TempFiles\\ICLR_CoDe\\stroke_gen')
            target = Image.open(target_path.joinpath(f'{target_dir.stem}.jpg'))

        ax[0][0].set_title('target')
        ax[0][0].imshow(target)
        ax[0][0].axis('off')
        for i in range(4):
            ax[i+1][0].axis('off')

        for j, method in enumerate(_METHODS_FACE.keys()):
            
            if task == 'styletransfer':
                curr_target = 'style_' + str(target_map[target_dir.stem]) if 'UG' in method else target_dir.stem
            else:
                curr_target = 'og_img_' + str(target_map[target_dir.stem]) if 'UG' in method else target_dir.stem

            # print(curr_target)
            # print(target_map[target_dir.stem])
            
            if 'Universal' in method:
                img_path = base_path.joinpath(_METHODS_FACE[method]).joinpath(curr_target).joinpath(prompt_dir.stem)
            else:
                img_path = base_path.joinpath(_METHODS_FACE[method]).joinpath(curr_target).joinpath('images').joinpath(prompt_dir.stem)

            # print(img_path)
            if not Path.exists(img_path):
                print(img_path)
                for i in range(5):
                    ax[i][j+1].axis('off')
                    if i == 0:
                        ax[i][j+1].set_title(method[:4])

                continue

            imgs = [x for x in img_path.iterdir() if Path.is_file(x)]
            # print(len(imgs))
            
            for i in range(5):
                if i < len(imgs):
                    ax[i][j+1].imshow(Image.open(imgs[i]))
                ax[i][j+1].axis('off')

                if i == 0:
                    ax[i][j+1].set_title(method)

        fig.suptitle(f'{target_dir.stem} : {prompt_dir.stem}')
        plt.tight_layout()
        # plt.show()

        # plt.show()
        print(plt_save)
        plt.savefig(plt_save.joinpath('res2.png'), dpi=300)
        plt.close()

    #     break
    # break

In [3]:
import json
import numpy as np

from pathlib import Path

In [ ]:
basedir = Path('D:/TempFiles/ICLR_CoDe/scorers/rewards_gram/rewards_gram/embeddings_c_code100_b5_r6_styletransfer')

overall = []
target_dirs = [x for x in basedir.iterdir() if Path.is_dir(x)]
for target_dir in target_dirs:
    prompt_dirs = [x for x in target_dir.iterdir() if Path.is_dir(x)]
    for prompt_dir in prompt_dirs:
        with open(prompt_dir.joinpath('reward_gram.json'), 'r') as fp:
            loaded_res = json.load(fp)
        
        overall.extend(loaded_res)

print(round(np.mean(overall),2))
print(round(np.std(overall),2))

# Non-cherry Picked Samples

In [2]:
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image

In [ ]:
base_path = Path('D:\TempFiles\ICLR_CoDe\i2i_r6\style_2\images')

prompts = [x for x in base_path.iterdir() if Path.is_dir(x)]
for p, prompt in enumerate(prompts):

    images = [x for x in prompt.iterdir()]

    fig, axes = plt.subplots(5, 5, figsize=(20,15))

    for i, (ax, img) in enumerate(zip(axes.flatten(), images)):
        # print(i)
        # print(images[i])
        ax.imshow(Image.open(img.as_posix()))
        ax.axis('off')
        # break
    
    plt.title(prompt.stem)
    plt.tight_layout()
    plt.savefig(f'i2i_{p}.png', dpi=300)
    # plt.show()
    break

In [9]:
_METHODS_FACE = {
    'BoN' : f'bon',
    'DPS (2023)' : f'dps',
    'UG (2024)': f'ug',
    'SVDD-PM (2024)': f'svdd',
    'SDEdit' : f'sdedit',
    'CoDe (Ours)' : f'code',
    'C-UG (2024)': f'ug_i2i',
    'C-CoDe (Ours)' : f'c_code',
}

base_path = Path('appendix')

for it in range(4):

    fig, ax = plt.subplots(12, 9, figsize=(15,25))

    target_path = Path('appendix/sdedit/style_2/images/target.png')
    target = Image.open(target_path)

    ax[0][0].set_title('target')
    ax[0][0].imshow(target)
    ax[0][0].axis('equal')
    ax[0][0].axis('off')

    for i in range(11):
        ax[i+1][0].axis('off')

    for j, method in enumerate(_METHODS_FACE.keys()):
        
        img_path = base_path.joinpath(_METHODS_FACE[method]).joinpath('style_2/A colorful photo of a eiffel tower')

        # print(img_path)
        if not Path.exists(img_path):
            print(img_path)
            for i in range(12):
                ax[i][j+1].axis('equal')
                ax[i][j+1].axis('off')
                if i == 0:
                    ax[i][j+1].set_title(method[:4])

            continue

        imgs = [x for x in img_path.iterdir() if Path.is_file(x)]
        # print(len(imgs))
        
        for i in range(12):
            if i < len(imgs):
                ax[i][j+1].imshow(Image.open(imgs[i+(it*5)]))
            ax[i][j+1].axis('equal')
            ax[i][j+1].axis('off')

            if i == 0:
                ax[i][j+1].set_title(method)

        # fig.suptitle(f'{target_dir.stem} : {prompt_dir.stem}')
        # plt.tight_layout()
        # plt.show()

    # plt.show()
    plt.tight_layout()
    # print(plt_save)
    plt.savefig(base_path.joinpath(f'res{it}_upd.png'), dpi=300)
    plt.close()

    # break